# Notebook for stencil validation and timing

Jan, Nina Horat & Laura Endres

In [ ]:
#Libraries
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import subprocess
import numpy as np

In [ ]:
#Name Lists
stencil_name_list = [
        "test",
        "laplacian1d", 
        "laplacian2d",
        "laplacian3d",
        "FMA",
       "lapoflap1d",
        "lapoflap2d",
        "lapoflap3d",
        "test_gt4py",
    ]

backend_list = ["numpy", 
                "numba_vector_function", 
                "numba_vector_decorator", 
                "numba_loop", 
                "numba_stencil", 
                "gt4py",
                "cupy",
                "numba_cuda"]

gt4py_backend_list = [
        "numpy", 
        "gtx86", 
        "gtmc", 
        "gtcuda"]


## Validation
### Create testfields
We create one testfield per stencil_name (option --create_field = True). After creation the fields are saved in the folder testfields and can be accessed by the option --field_name.

In [ ]:
#comment out if you want to create new validation fields
# for x in stencil_name_list:
    
#     bashCommand=f"python3 stencil_main_validation.py --nx 32 --ny 32 --nz 32 --stencil_name {x} --backend numpy --create_field True --field_name {x}"
#     process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE,stderr=subprocess.PIPE)
#     output, error = process.communicate()
    
#     print(output.decode("utf-8"))

In [ ]:
for x in stencil_name_list:
    print(x,':')
    for y in backend_list:
        
        if y=='gt4py':
            if x=='test':
                print('There is no test stencil in gt4py.')
            else:
                for z in gt4py_backend_list:
                    bashCommand=f"python3 stencil_main_validation.py --nx 32 --ny 32 --nz 32 --stencil_name {x} --backend {y} --gt4py_backend {z} --create_field False --field_name {x}"
                    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
                    output, error = process.communicate()
                    print(output.decode("utf-8"),' for ', z)
                
        else:
            bashCommand=f"python3 stencil_main_validation.py --nx 32 --ny 32 --nz 32 --stencil_name {x} --backend {y} --create_field False --field_name {x}"
            process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()
            print(output.decode("utf-8"))

## Timing
### Execute Stencil computation for evaluation
In this section the different stencils for the different domain sizes are calculated.


In [ ]:
#this takes some time...
#Parameter 
field_size_list = [128,256]
df_name = "val" #Name of evaluation dataframe
num_iter = 20

#Stencil computation
for size in field_size_list:
    print(size,':')
    for x in stencil_name_list:
        print('    ',x,':')
        for y in backend_list:
            bashCommand=f"python3 stencil_main_performance.py --nx {size} --ny {size} --nz {size} --stencil_name {x} --backend {y} --num_iter {num_iter} --df_name {df_name}"
            process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
            print('                 ',y,' calculated.')
            output, error = process.communicate()
            #print(output.decode("utf-8"))

### Read out the dataframe and plot results
The program saves the evaluation data into a pandas dataframe (Name is defined with the option --df_name). Subsequently the df can be accessed for further processing within a jupyter notebook.

In [ ]:
df_name = "val_N" #Name of df to evaluate
df = pd.read_pickle("eval/{}_result.pkl".format(df_name))
df.tail(20)

In [ ]:
df_name = "val_128" #Name of df to evaluate
df = pd.read_pickle("eval/{}_result.pkl".format(df_name))

for size in field_size_list:
        df_plot = df.loc[df["nx"]==size].sort_values(by=["stencil_name"])
        plt.figure(figsize=(10,5))
        chart=sns.barplot(x="stencil_name", y="run_avg", hue="backend",data=df_plot, ci=False)
        chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
        chart.legend(loc='upper left')
        chart.set_title('Runtimes for stencil computation for nx,ny,nz={}'.format(size))
        chart.set_yscale("log")


## Data analysis:

In [ ]:
df1 = pd.read_pickle("eval/{}_result.pkl".format("val_16_32_64"))
df2 = pd.read_pickle("eval/{}_result.pkl".format("val_128_256_512_1024"))  
#df3 = pd.read_pickle("eval/{}_result.pkl".format(df_name)) 

frames = [df1, df2]
df_data = pd.concat(frames)


In [ ]:
df=df_data.loc[df_data["nx"]==64]
#print(df)
df.info()

### for different stencils

In [ ]:
for name in stencil_name_list:
    data_stencil = df_data.loc[df_data["stencil_name"] == name]
    #print(data_stencil.head(20))
    plt.figure(figsize=(10,5))
    chart=sns.catplot(x="nx", y="run_avg", 
                      hue="backend",
                      data=data_stencil,ci=False,
                      s=14
                     )
    plt.title('Runtimes for stencil {}'.format(name))
    #chart.set_yscale("log")###not possible with catplot

In [ ]:
for name in stencil_name_list:
    data_stencil = df_data.loc[df_data["stencil_name"] == name].sort_values(by=["backend"])
    #print(data_stencil.head(20))
    plt.figure(figsize=(10,5))
    chart=sns.pointplot(x="nx", y="run_avg", 
                      hue="backend",
                      data=data_stencil,ci=False
                     )
    #chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
    #chart.legend(loc='upper left')
    plt.title('Runtimes for stencil {}'.format(name))
    #chart.set_title('Runtimes for stencil {}'.format(name))
    chart.set_yscale("log")

In [ ]:
plt.close('all')

In [ ]:
print(df_data.head())
#df_data.info()
data_stencil = df_data.groupby(['nx','backend']).mean()
data_stencil.reset_index(inplace=True)#.unstack(level=-1)#.unstack(level=0)
data_stencil.head()

In [ ]:
#average over stencils - figure I would put in report
data_stencil = df_data.groupby(['nx','backend']).mean()
data_stencil.reset_index(inplace=True)
#print(data_stencil.head(20))
plt.figure(figsize=(10,5))
chart=sns.pointplot(x="nx", y="run_avg", 
                  hue="backend",
                  data=data_stencil,ci=False
                 )
chart.set(xlabel='field size', ylabel='runtime [s]')
plt.title('Runtime averaged over all stencils')
chart.set_yscale("log")

## per field size:

In [ ]:
for size in df_data.nx.unique():
    data_field_size = df_data.loc[df_data["nx"] == size].sort_values(by=["backend","stencil_name"])
    #print(data_field_size.head(20))
    plt.figure(figsize=(10,5))
    chart=sns.pointplot(x="stencil_name", y="run_avg", hue="backend",data=data_field_size, ci=False)
    chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
    chart.legend(loc='upper left')
    chart.set_title('Runtimes for stencil computation for nx,ny,nz={}'.format(size))
    chart.set_yscale("log")

In [ ]:

df_name = "val_1" #Name of df to evaluate
df = pd.read_pickle("eval/{}_result.pkl".format(df_name))
df.tail(10)

df16=df.loc[df["nx"]==16]
plt.figure(figsize=(10,5))
chart=sns.barplot(x="stencil_name", y="run_avg", hue="backend",data=df16, ci=False)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
chart.legend(loc='upper left')


df32=df.loc[df["nx"]==32]
plt.figure(figsize=(10,5))
chart=sns.barplot(x="stencil_name", y="run_avg", hue="backend",data=df32, ci=False)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
chart.legend(loc='upper left')

df64=df.loc[df["nx"]==64]
plt.figure(figsize=(10,5))
chart=sns.barplot(x="stencil_name", y="run_avg", hue="backend",data=df64, ci=False)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
chart.legend(loc='upper left')

In [ ]:
df_name = "val_N" #Name of df to evaluate
df = pd.read_pickle("eval/{}_result.pkl".format(df_name))
df.tail(10)

df16=df.loc[df["nx"]==128]
plt.figure(figsize=(10,5))
chart=sns.barplot(x="stencil_name", y="run_avg", hue="backend",data=df16, ci=False)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
chart.legend(loc='upper left')

In [ ]:
#############first try to add error bars. 
#the location of the bars is still wrong since the rows get ordered differently when .groupby is applied. 
#also I had to add data points for gt4py and numba_stencil where the function do not yet work.
#but this would be an option to get the error bars on top of everything.
#https://stackoverflow.com/questions/62820959/use-precalculated-error-bars-with-seaborn-and-barplot
#https://matplotlib.org/gallery/lines_bars_and_markers/errorbar_limits_simple.html#sphx-glr-gallery-lines-bars-and-markers-errorbar-limits-simple-py

df_name = "val_1" #Name of df to evaluate
df = pd.read_pickle("eval/{}_result.pkl".format(df_name))
df.tail(10)

df16=df.loc[df["nx"]==16]
plt.figure(figsize=(10,5))
chart =sns.barplot(x="stencil_name", y="run_avg", hue="backend",data=df16, ci=False)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
chart.legend(loc='upper left')

conc2=[0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5,6,6,6,6,6,6,7,7,7,7,7,7]
width = .25
add = [-2.5*width, -1.5*width, -0.5*width, 0.5*width , 1.5*width, 2.5*width, -2.5*width, -1.5*width, -0.5*width, 0.5*width , 1.5*width, 2.5*width,
       -2.5*width, -1.5*width, -0.5*width, 0.5*width , 1.5*width, 2.5*width,-2.5*width, -1.5*width, -0.5*width, 0.5*width , 1.5*width, 2.5*width,
       -2.5*width, -1.5*width, -0.5*width, 0.5*width , 1.5*width, 2.5*width,-2.5*width, -1.5*width, -0.5*width, 0.5*width , 1.5*width, 2.5*width,
       -2.5*width, -1.5*width, -0.5*width, 0.5*width , 1.5*width, 2.5*width,-2.5*width, -1.5*width, -0.5*width, 0.5*width , 1.5*width, 2.5*width,
      ]
x = np.array(conc2)+np.array(add)
#print(x)

df16_stdev=df16.groupby(['stencil_name','backend']).mean()
df16_stdev=df16_stdev.append(df16_stdev[-4:-1])
#print(df16_stdev)
print(df16_stdev.shape)
plt.errorbar(x = x, y = df16_stdev['run_avg'],
            yerr = df16_stdev['run_stdev'],
             fmt='none', c= 'black', capsize = 2
            )
plt.show()


In [ ]:
#Clear df on disk
df_name = "test"
os.remove("eval/{}_result.pkl".format(df_name))

### Evaluate Runtime
We suspect that different number of iterations will lead to different runtime developments.
This can be tested with the option --save_runtime. The df runtimedevelopment can afterwards be evaluated.

In [ ]:
df_runtime = pd.read_pickle("eval/runtimedevelopment.pkl")
df_runtime.columns = ['runtime']
df_runtime.info()

In [ ]:
df_runtime.plot()
#numba_loop 32x32x32 stencil: lapoflap3d

In [ ]:
df_runtime.plot()
#numba_loop 64x64x64 stencil: lapoflap3d

In [ ]:
df_runtime.plot()
#numba_vector_function 64x64x64 stencil: lapoflap3d